Iman Conover procedure

In [430]:
from pyDOE import *
from scipy import stats as sct
from scipy import linalg
import matplotlib.pyplot as plt
import pandas as pd


n = desired no. samples
k = no. parameters
dist_in = details of input distributions

In [439]:
dist_in =  pd.DataFrame(np.array([['Var01', 'uniform',2, 3, 0], ['Var02', 'normal',2.5, 0.5, 0],['Var03', 'triangular',2, 8, 3],['Var04', 'normal',2, 1, 3],['Var05', 'triangular',2, 5, 3],['Var06', 'uniform',2, 4, 3]]),
...                    columns=['Varname', 'dist_type', 'dist_param_1','dist_param_2','dist_param_3'])
k = dist_in.shape[0]
n = 800



In [440]:
design = lhs(k,samples = n)
#design = design.transpose()
print(design.shape)

(800, 6)


In [441]:
for i in iter(range(k)):
    print(i)
    a= float(dist_in.loc[i,'dist_param_1'])*np.ones((n,), dtype=int)
    b= float(dist_in.loc[i,'dist_param_2'])*np.ones((n,), dtype=int)
    d= float(dist_in.loc[i,'dist_param_3'])*np.ones((n,), dtype=int)
    c = (d/(b-a))

  
    if dist_in.loc[i,'dist_type']=='normal':
        design[:, i] = sct.norm(loc =a, scale=b).ppf(design[:,i])
    if dist_in.loc[i,'dist_type']=='uniform':
         design[:, i] = sct.uniform(loc =a, scale=(b-a)).ppf(design[:,i])
    if dist_in.loc[i,'dist_type']=='triangular':
        design[:, i] = sct.triang(loc =a, scale=(b-a), c=c ).ppf(design[:,i])
  

0
1
2
3
4
5


cor = correlation matrix

Calculate correlation matrix T for input data 

In [442]:
cor =np.array([[1,0,0,0,0,0],[0,1,0,0,0,0],[0,0,1,0,0,0],[0,0,0,1,0.75,-0.7],[0,0,0,0.75,1,-0.95],[0,0,0,-.70,-0.95,1]])
T = pd.DataFrame(design).corr()
print(cor)


[[ 1.    0.    0.    0.    0.    0.  ]
 [ 0.    1.    0.    0.    0.    0.  ]
 [ 0.    0.    1.    0.    0.    0.  ]
 [ 0.    0.    0.    1.    0.75 -0.7 ]
 [ 0.    0.    0.    0.75  1.   -0.95]
 [ 0.    0.    0.   -0.7  -0.95  1.  ]]


Use cholesky decomposition to select lower triangular matrix where  $LL^{-1} = C $

Use cholesky decomposition to select lower triangular matrix where  $QQ^{-1} = T $

Calculate S as $L *Q^{-1} $

Multiply the samples by L to induce the correlations and by $S\prime$ to remove the effects of unintended correlations in the input sample.

In [443]:
L = linalg.cholesky(cor, lower=True)
Q = linalg.cholesky(T, lower=True)
S = L@np.linalg.inv(Q)
ranked = design@S.transpose()


Rank the columns of the resulting matrix and sort the columns of the original sample matrix to match the rank order

In [447]:
rank = np.zeros((design.shape[0], design.shape[1]))

for j in range(0, design.shape[1]):
    rank[:, j] = sct.rankdata(ranked[:, j], method='ordinal')
rank_order = rank.astype(int) - 1

ranked_samples = np.zeros((design.shape[0], design.shape[1]))
for m in range(0, k):
    s = np.sort(design[:, m])
    ranked_samples[:, m] = s[rank_order[:,m]]
   
 


In [448]:
ranked_df=pd.DataFrame(ranked_samples)


Print the correlation matrix for the ranked sample to compare with the intended correlation matrix, cor.

In [449]:
check_matrix = ranked_df.corr()
print(check_matrix)

          0         1         2         3         4         5
0  1.000000 -0.013815  0.053646  0.022691  0.019942 -0.009088
1 -0.013815  1.000000 -0.008434 -0.022563 -0.024076  0.029576
2  0.053646 -0.008434  1.000000  0.009931  0.011388 -0.021738
3  0.022691 -0.022563  0.009931  1.000000  0.817778 -0.801303
4  0.019942 -0.024076  0.011388  0.817778  1.000000 -0.957678
5 -0.009088  0.029576 -0.021738 -0.801303 -0.957678  1.000000
